In [ ]:
import numpy as np
import pandas as pd 

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

plt.style.use("ggplot")
sns.set_theme()
pd.set_option("display.max_columns",None)

## Chargement du dataset et appercu general

In [ ]:
#charge le dataset
df = pd.read_csv("netflix_titles.csv")

print("Shape (lignes,colonnes) :",df.shape)
display(df.head())

df.info()

df.describe(include="all").T

## Valeurs manquantes et doublons

In [ ]:
#Valeurs manquantes par colonnes
df.isna().sum()

In [ ]:
#pourcentage de valeurs manquantes par colonnes
(df.isna().mean()*100).round(2)

In [ ]:
#Doublons
print("Nombres de lignes dupliquee :",df.duplicated().sum())

## Nettoyage

In [ ]:
#S'occupe des dates pas formatte pareill.
#Split en 2 colonnes; date added en datetime , year added et month added.
df["date_added"] = pd.to_datetime(df["date_added"],
    format="mixed"
    )
df["year_added"] = df["date_added"].dt.year
df["month_added"] = df["date_added"].dt.month

print(df[["date_added", "year_added", "month_added"]].head())

In [ ]:
#extrait la partie int de la duration.
df["duration_int"] = (
    df["duration"].str.extract(r"(\d+)").astype(float)
)

#visualisation et check
df[["type","duration","duration_int"]].head()

In [ ]:
display(df.head())

## analyse des contenus

### Films vs séries : proportions, tendances par année

### Genre principaux

### Repartition geographique

In [ ]:
 #1. Compter les valeurs manquantes
print(f"Valeurs manquantes dans 'country': {df['country'].isna().sum()} ({(df['country'].isna().mean()*100):.2f}%)")
print()

# 2. Séparer les pays multiples et créer une liste de tous les pays
# Utiliser explode pour transformer chaque pays en ligne séparée
df_country = df['country'].dropna().str.split(',', expand=False)
all_countries = df_country.explode().str.strip()
print(f"Nombre total de titres avec pays: {len(df_country)}")
print(f"Nombre total d'associations titre-pays: {len(all_countries)}")
print()
country_counts = all_countries.value_counts().reset_index()
country_counts.columns = ['country', 'count']

# 1. Bar plot horizontal - Top 15 pays (Seaborn)
plt.figure(figsize=(12, 8))
top_15_countries = all_countries.value_counts().head(15)
sns.barplot(x=top_15_countries.values, y=top_15_countries.index, palette='viridis')
plt.xlabel('Nombre de médias', fontsize=12)
plt.ylabel('Pays', fontsize=12)
plt.title('Top 15 des pays par contenu sur Netflix', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

# 2. Pie chart - Top 10 pays (pour voir les proportions)
plt.figure(figsize=(10, 8))
top_10_countries = all_countries.value_counts().head(10)
colors = sns.color_palette('pastel')[0:10]
plt.pie(top_10_countries.values, 
        labels=top_10_countries.index, 
        autopct='%1.1f%%',
        startangle=90,
        colors=colors)
plt.title('Distribution des Top 10 pays producteurs', fontsize=14, fontweight='bold')
plt.axis('equal')
plt.show()

# 3. Heatmap - Répartition des pays par type de contenu (Movie vs TV Show)

# Créer une crosstab entre pays et type de contenu
# D'abord, on crée un DataFrame temporaire avec pays et type
df_temp = df[['type', 'country']].copy()
df_temp = df_temp.dropna()
df_temp = df_temp.assign(country=df_temp['country'].str.split(',')).explode('country')
df_temp['country'] = df_temp['country'].str.strip()

# Créer la crosstab (pivot table) : pays en ligne, types en colonne
country_type_matrix = pd.crosstab(df_temp['country'], df_temp['type'])

# Sélectionner les Top 15 pays pour une meilleure lisibilité
top_15_countries_list = all_countries.value_counts().head(15).index.tolist()
country_type_matrix_top = country_type_matrix.loc[country_type_matrix.index.isin(top_15_countries_list)]

# Créer la heatmap
plt.figure(figsize=(10, 10))
sns.heatmap(country_type_matrix_top, 
            annot=True,  # Afficher les valeurs dans les cellules
            fmt='d',  # Format entier
            cmap='YlOrRd',  # Palette de couleurs
            cbar_kws={'label': 'Nombre de contenus'},
            linewidths=0.5,
            linecolor='gray')
plt.title('Répartition des contenus Netflix par pays et type', fontsize=14, fontweight='bold')
plt.xlabel('Type de contenu', fontsize=12)
plt.ylabel('Pays', fontsize=12)
plt.tight_layout()
plt.show()

### Casting & réalisateurs : analyse simple (comptages, noms fréquents)

In [ ]:
## Casting & Réalisateurs : analyse simple (comptages, noms fréquents)

### Analyse des Réalisateurs

# 1. Statistiques de base sur les réalisateurs
print("=== ANALYSE DES RÉALISATEURS ===")
print(f"Valeurs manquantes dans 'director': {df['director'].isna().sum()} ({(df['director'].isna().mean()*100):.2f}%)")
print()

# 2. Séparer les réalisateurs multiples
df_directors = df['director'].dropna().str.split(',', expand=False)
all_directors = df_directors.explode().str.strip()

print(f"Nombre total de titres avec réalisateur: {len(df_directors)}")
print(f"Nombre total d'associations titre-réalisateur: {len(all_directors)}")
print(f"Nombre de réalisateurs uniques: {all_directors.nunique()}")
print()

# 3. Top 20 réalisateurs les plus prolifiques
top_directors = all_directors.value_counts().head(20)
print("Top 20 des réalisateurs les plus prolifiques sur Netflix:")
print(top_directors)
print()

# 4. Créer un DataFrame pour les réalisateurs
director_counts = all_directors.value_counts().reset_index()
director_counts.columns = ['director', 'count']

# 5. Analyse par type de contenu
print("\n--- Réalisateurs par type de contenu ---")
for content_type in df['type'].unique():
    df_type = df[df['type'] == content_type]
    directors_type = df_type['director'].dropna().str.split(',', expand=False).explode().str.strip()
    top_5 = directors_type.value_counts().head(5)
    print(f"\nTop 5 réalisateurs pour {content_type}:")
    print(top_5)

print("\n" + "="*80)

### Analyse du Casting

print("\n=== ANALYSE DU CASTING ===")

# 1. Statistiques de base sur le casting
print(f"Valeurs manquantes dans 'cast': {df['cast'].isna().sum()} ({(df['cast'].isna().mean()*100):.2f}%)")
print()

# 2. Séparer les acteurs multiples
df_cast = df['cast'].dropna().str.split(',', expand=False)
all_actors = df_cast.explode().str.strip()

print(f"Nombre total de titres avec casting: {len(df_cast)}")
print(f"Nombre total d'associations titre-acteur: {len(all_actors)}")
print(f"Nombre d'acteurs uniques: {all_actors.nunique()}")
print()

# 3. Top 20 acteurs les plus présents
top_actors = all_actors.value_counts().head(20)
print("Top 20 des acteurs les plus présents sur Netflix:")
print(top_actors)
print()

# 4. Créer un DataFrame pour les acteurs
actor_counts = all_actors.value_counts().reset_index()
actor_counts.columns = ['actor', 'count']

# 5. Analyse par type de contenu
print("\n--- Acteurs par type de contenu ---")
for content_type in df['type'].unique():
    df_type = df[df['type'] == content_type]
    actors_type = df_type['cast'].dropna().str.split(',', expand=False).explode().str.strip()
    top_5 = actors_type.value_counts().head(5)
    print(f"\nTop 5 acteurs pour {content_type}:")
    print(top_5)

# 6. Statistiques supplémentaires
print("\n--- Statistiques supplémentaires ---")
avg_cast_size = df_cast.apply(len).mean()
print(f"Nombre moyen d'acteurs par titre: {avg_cast_size:.2f}")

avg_directors = df_directors.apply(len).mean()
print(f"Nombre moyen de réalisateurs par titre: {avg_directors:.2f}")

### VISUALISATIONS POUR RÉALISATEURS ET ACTEURS

# 1. Bar plot horizontal - Top 15 réalisateurs (Seaborn)
plt.figure(figsize=(12, 8))
top_15_directors = all_directors.value_counts().head(15)
sns.barplot(x=top_15_directors.values, y=top_15_directors.index, palette='rocket')
plt.xlabel('Nombre de contenus', fontsize=12)
plt.ylabel('Réalisateur', fontsize=12)
plt.title('Top 15 des réalisateurs les plus prolifiques sur Netflix', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

# 2. Bar plot horizontal - Top 15 acteurs (Seaborn)
plt.figure(figsize=(12, 8))
top_15_actors = all_actors.value_counts().head(15)
sns.barplot(x=top_15_actors.values, y=top_15_actors.index, palette='mako')
plt.xlabel('Nombre de contenus', fontsize=12)
plt.ylabel('Acteur', fontsize=12)
plt.title('Top 15 des acteurs les plus présents sur Netflix', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

# 3. Comparaison côte à côte - Top 10 réalisateurs vs Top 10 acteurs
fig, axes = plt.subplots(1, 2, figsize=(16, 8))

# Réalisateurs
top_10_directors = all_directors.value_counts().head(10)
axes[0].barh(range(len(top_10_directors)), top_10_directors.values, color='skyblue')
axes[0].set_yticks(range(len(top_10_directors)))
axes[0].set_yticklabels(top_10_directors.index)
axes[0].set_xlabel('Nombre de contenus')
axes[0].set_title('Top 10 Réalisateurs', fontweight='bold')
axes[0].invert_yaxis()

# Acteurs
top_10_actors = all_actors.value_counts().head(10)
axes[1].barh(range(len(top_10_actors)), top_10_actors.values, color='lightcoral')
axes[1].set_yticks(range(len(top_10_actors)))
axes[1].set_yticklabels(top_10_actors.index)
axes[1].set_xlabel('Nombre de contenus')
axes[1].set_title('Top 10 Acteurs', fontweight='bold')
axes[1].invert_yaxis()

plt.suptitle('Comparaison des contributeurs Netflix les plus prolifiques', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()



## Analyse Temporelle

### Distribution par annee

### Analyse de la colonne date_added.

## Visualistion Interactive

### Représentations obligatoires :

### histogrammes, countplots, boxplots (Seaborn / Matplotlib)
### visualisations interactives (Plotly)

### Choix libres : nuages de mots, diagrammes circulaires, timelines, etc.

## Synthèse


### Résumé des observations principales.

### Tendances remarquables (contenus récents, pays dominants, genrespopulaires).
